In [1]:
from pathlib import Path

import geopandas as gpd
import icechunk
import numpy as np
import pandas as pd
import xarray as xr
import zarr
from icechunk.xarray import to_icechunk
from tqdm.notebook import tqdm
from virtualizarr import open_virtual_dataset

In [2]:
import boto3
import os

profile_name = "CIROH_USER"  # Replace with your AWS credentials file profile name
os.environ['AWS_PROFILE'] = profile_name

In [ ]:
file_path = Path("/projects/mhpi/yxs275/DM_output/HydroFabric_forward_1980_2019_From_dPL_local_daymet_v6_2v18_2_oneGPU_dynamic_k0_1980_1995")
bucket="mhpi-spatial"
prefix="hydrofabric_v2.2_dhbv_retrospective"
commit="initial streamflow commit"

### Reading the icechunk store

In [4]:
storage_config = icechunk.s3_storage(
bucket=bucket, prefix=prefix, region="us-east-2", anonymous=True
)
repo = icechunk.Repository.open(storage_config)
session = repo.readonly_session("main")
cloud_ds = xr.open_zarr(session.store, consolidated=False)

xr.set_options(display_style="text")
cloud_ds

/home/tbindas/tbindas/.conda/envs/nwm_batcher/lib/python3.12/site-packages/zarr/codecs/vlen_utf8.py:44: UserWarning: The codec `vlen-utf8` is currently not part in the Zarr format 3 specification. It may not be supported by other zarr implementations and may change in the future.
  return cls(**configuration_parsed)


<xarray.Dataset> Size: 24GB
Dimensions:    (time: 14610, divide_id: 409329)
Coordinates:
  * time       (time) datetime64[ns] 117kB 1980-01-01 1980-01-02 ... 2019-12-31
  * divide_id  (divide_id) object 3MB 'cat-2872054' ... 'cat-837052'
Data variables:
    Qr         (divide_id, time) float32 24GB dask.array<chunksize=(188, 1827), meta=np.ndarray>
Attributes:
    description:  Runoff outputs from dhbv2.0 at the HFv2.2 catchment scale

In [3]:
gdf = gpd.read_file("/projects/mhpi/data/hydrofabric/v2.2/conus_nextgen.gpkg", layer="divides")
gdf = gdf.set_index("divide_id")

In [4]:
np.unique(gdf.index).shape

(831777,)

In [33]:
root = zarr.open_group("/projects/mhpi/yxs275/DM_output/HydroFabric_forward_1980_2019_From_dPL_local_daymet_v6_2v18_2_oneGPU_dynamic_k0_1980_1995")
divide_ids = []
for key in root.keys():
    _root = root[key]
    divide_ids.extend(_root["divide_id"][:].tolist())
divide_ids = np.array(divide_ids)
np.unique(divide_ids).shape

(637309,)

In [19]:
gdf.loc[~gdf.index.isin(divide_ids)].reset_index().to_file(
    "/projects/mhpi/tbindas/CIROH_DL_NextGen/icechunk_scripts/missing.gpkg"
)

In [30]:
count = 0
for file in all_files:
    count += 1
    
count

0

In [ ]:
# From HF v2.2
np.unique(gdf.index)

array(['cat-1', 'cat-10', 'cat-100', ..., 'cat-99997', 'cat-99998',
       'cat-99999'], shape=(831777,), dtype=object)

In [ ]:
# From dHBV2.0 forwarding
np.unique(divide_ids)

array(['cat-100', 'cat-1000', 'cat-10000', ..., 'cat-99997', 'cat-99998',
       'cat-99999'], shape=(637309,), dtype='<U11')